In [ ]:
import sys
from random import random

from builtins import chr

from PIL import Image, ImageDraw, ImageFont

In [ ]:
font_text  = ImageFont.truetype('/Users/marcuspompeu/Library/Fonts/Roboto-Bold.ttf', size=32)
font_arrow = ImageFont.truetype('/Users/marcuspompeu/Library/Fonts/Wingdings 3.ttf', size=32)

In [ ]:
image_canvas_logo = Image.open('./Canvas Logo.png', 'r')

In [ ]:
kpi_active_students = 2345322
kpi_submissions     = 3104122
kpi_uptime          = 0.9932
kpi_support_tickets = 12221
kpi_overdue_tickets = 0.0010
kpi_ff_flag         = 0.3200

kpi_good_if_raising = [True, True, True, False, False, True]

texts = \
    {
        t:
            {
                'length': font_text.getlength(t),
                'height': font_text.getbbox(t)[3] - font_text.getbbox(t)[1] + 1
            }
        for t
        in
            [
                f'Monthly Active Students: {kpi_active_students:,}',
                f'Weekly Submissions: {kpi_submissions:,}',
                f'Average Uptime: {kpi_uptime:.2%}',
                f'New Weekly Support Tickets: {kpi_support_tickets:,}',
                f'Weekly Overdue Support Tickets: {kpi_overdue_tickets:.2%}',
                f'Improved Outcomes Feature Flag: {kpi_ff_flag:.2%}'
            ]
    }

In [ ]:
width_logo  = image_canvas_logo.width
margin_logo = 16
#
width_arrow = font_arrow.getlength(chr(112))
#
spacer_group  = 32
spacer_ticker = 16
width_group_spacer  = 32
width_ticker_spacer = 16


width_total = \
    int(
        2 +
        margin_logo + width_logo + margin_logo +
        sum(
            [
                d['length'] + spacer_ticker + width_arrow + spacer_group
                for d
                in texts.values()
            ]
        )
    )

In [ ]:
height_logo      = image_canvas_logo.height
height_arrow     = font_arrow.getbbox(chr(112))[3] - font_arrow.getbbox(chr(112))[1] + 1
height_max_texts = max(d['height'] for d in texts.values())

In [ ]:
ticker_height = 96

image      = Image.new('RGBA', (width_total, 96), color='black')
image_draw = ImageDraw.Draw(image)

def do_draw():
    def do_canvas_logo():
        nonlocal col
        
        image.paste(image_canvas_logo, (2 + margin_logo, (ticker_height - height_logo) // 2))
        
        col += margin_logo + image_canvas_logo.width + margin_logo        

    def do_draw_text(text, height, font, fill=None):
        image_draw.text((col, (ticker_height - height) // 2), text, font=font, fill=fill)
        
    def do_text(item):
        nonlocal col
        
        text   = item[0]
        length = item[1]['length']
        height = item[1]['height']
        
        do_draw_text(text, height, font_text)
        
        col += length + spacer_ticker

    def do_ticker(idx):
        nonlocal col

        up_or_down = round(random())
        up_or_down_color = (0, 172, 24) if kpi_good_if_raising[idx] and not up_or_down else (238, 6, 18)

        do_draw_text(chr(112 + up_or_down), height_arrow, font_arrow, up_or_down_color)
        
    def do_group_spacer(idx):
        nonlocal col
        
        if idx != len(texts) - 1:
            col += width_arrow + spacer_group // 2

            image_draw.rectangle((col, (image.height - height_max_texts) // 2, col + 1, (image.height - height_max_texts) // 2 + height_max_texts), fill=(255, 255, 255))

            col += spacer_group // 2

    col = 2
    
    do_canvas_logo()    

    for i, item in enumerate(texts.items()):
        do_text(item)
        do_ticker(i)
        do_group_spacer(i)


do_draw()    

In [ ]:
image_draw.rectangle((0,               0,                1,               image.height),     fill=(255, 255, 255))
image_draw.rectangle((0,               0,                image.width,     1),                fill=(255, 255, 255))
image_draw.rectangle((0,               image.height - 2, image.width,     image.height - 1), fill=(255, 255, 255))
image_draw.rectangle((image.width - 2, 0,                image.width - 1, image.height),     fill=(255, 255, 255))